### Imports

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from math import pi
import plotly.express as px

CSV loading

In [ ]:
economy = pd.read_csv("../data/economy_data.csv")
communications = pd.read_csv("../data/communications_data.csv")
demographics = pd.read_csv("../data/demographics_data.csv")
energy = pd.read_csv("../data/energy_data.csv")
geography = pd.read_csv("../data/geography_data.csv")
governmentcivics = pd.read_csv("../data/government_and_civics_data.csv")
transportation = pd.read_csv("../data/transportation_data.csv")

Cleaning and merging

In [ ]:
import pandas as pd
import numpy as np

# ------------------------------------------------------------
# Helper: clean numeric-looking columns (remove commas, %, etc.)
# ------------------------------------------------------------
def clean_numeric(series: pd.Series) -> pd.Series:
    """
    Take a pandas Series that should be numeric (but may contain commas,
    percent signs, 'NA', etc.) and return a Series of cleaned strings.
    """
    return (
        series.astype(str)
        .str.replace(",", "", regex=False)
        .str.replace("%", "", regex=False)
        .str.replace("−", "-", regex=False)  # sometimes minus is a different unicode char
        .replace({"nan": np.nan, "NA": np.nan, "": np.nan})
    )

# ------------------------------------------------------------
# 1) Load all CSVs
#    (change to "../data/..." if your notebook is in /notebooks)
# ------------------------------------------------------------

economy = pd.read_csv("../data/economy_data.csv")
comms = pd.read_csv("../data/communications_data.csv")
demo = pd.read_csv("../data/demographics_data.csv")
energy = pd.read_csv("../data/energy_data.csv")
geo = pd.read_csv("../data/geography_data.csv")
gov = pd.read_csv("../data/government_and_civics_data.csv")
trans = pd.read_csv("../data/transportation_data.csv")

# ------------------------------------------------------------
# 2) Define which columns are numeric in each table
#    (so we don't destroy text like Government_Type, Capital, etc.)
# ------------------------------------------------------------

eco_num_cols = [
    "Real_GDP_PPP_billion_USD",
    "GDP_Official_Exchange_Rate_billion_USD",
    "Real_GDP_Growth_Rate_percent",
    "Real_GDP_per_Capita_USD",
    "Unemployment_Rate_percent",
    "Youth_Unemployment_Rate_percent",
    "Budget_billion_USD",
    "Budget_Surplus_billion_USD",
    "Budget_Deficit_percent_of_GDP",
    "Public_Debt_percent_of_GDP",
    "Exports_billion_USD",
    "Imports_billion_USD",
    "Exchange_Rate_per_USD",
    "Population_Below_Poverty_Line_percent",
]

demo_num_cols = [c for c in demo.columns if c != "Country"]

energy_num_cols = [
    "electricity_access_percent",
    "electricity_generating_capacity_kW",
    "coal_metric_tons",
    "petroleum_bbl_per_day",
    "refined_petroleum_products_bbl_per_day",
    "refined_petroleum_exports_bbl_per_day",
    "refined_petroleum_imports_bbl_per_day",
    "natural_gas_cubic_meters",
    "carbon_dioxide_emissions_Mt",
]

trans_num_cols = [
    "airports_paved_runways_count",
    "airports_unpaved_runways_count",
    "heliports_count",
    "roadways_km",
    "railways_km",
    "waterways_km",
    "gas_pipelines_km",
    "oil_pipelines_km",
    "refined_products_pipelines_km",
    "water_pipelines_km",
]

comms_num_cols = [
    "telephone_fixed_subscriptions_total",
    "mobile_cellular_subscriptions_total",
    "internet_users_total",
    "broadband_fixed_subscriptions_total",
]

# In geography, some columns are textual (Land_Boundaries, Coastline),
# others are numeric:
geo_num_cols = [
    "Area_Total",
    "Land_Area",
    "Water_Area",
    "Highest_Elevation",
    "Lowest_Elevation",
    "Forest_Land",
    "Other_Land",
    "Agricultural_Land",
    "Arable_Land (%% of Total Agricultural Land)",
    "Permanent_Crops (%% of Total Agricultural Land)",
    "Permanent_Pasture (%% of Total Agricultural Land)",
    "Irrigated_Land",
]

gov_num_cols = ["Suffrage_Age"]

# ------------------------------------------------------------
# 3) Clean numeric columns in each dataframe
#    (we keep all rows; we just ensure types are clean)
# ------------------------------------------------------------

for col in eco_num_cols:
    economy[col] = pd.to_numeric(clean_numeric(economy[col]), errors="coerce")

for col in demo_num_cols:
    if col != "Country":
        demo[col] = pd.to_numeric(clean_numeric(demo[col]), errors="coerce")

for col in energy_num_cols:
    energy[col] = pd.to_numeric(clean_numeric(energy[col]), errors="coerce")

for col in trans_num_cols:
    trans[col] = pd.to_numeric(clean_numeric(trans[col]), errors="coerce")

for col in comms_num_cols:
    comms[col] = pd.to_numeric(clean_numeric(comms[col]), errors="coerce")

for col in geo_num_cols:
    geo[col] = pd.to_numeric(clean_numeric(geo[col]), errors="coerce")

for col in gov_num_cols:
    gov[col] = pd.to_numeric(clean_numeric(gov[col]), errors="coerce")

# ------------------------------------------------------------
# 4) Merge all tables on Country
#    Use OUTER joins so we NEVER drop any country.
# ------------------------------------------------------------

df = economy.copy()

df = df.merge(demo,   on="Country", how="outer")
df = df.merge(energy, on="Country", how="outer")
df = df.merge(trans,  on="Country", how="outer")
df = df.merge(comms,  on="Country", how="outer")
df = df.merge(geo,    on="Country", how="outer")
df = df.merge(gov,    on="Country", how="outer")

# ------------------------------------------------------------
# 5) Fill missing numeric values with 0.0
#    Justification:
#    - For an interactive dashboard, it's often more practical
#      to treat missing infrastructure quantities as 0 in
#      charts (no errors, clear "no data / no infrastructure").
#    - If later you need to distinguish true zero vs missing,
#      you can keep a copy of the raw merged file or use masks.
# ------------------------------------------------------------

numeric_cols = df.select_dtypes(include=["number"]).columns
df[numeric_cols] = df[numeric_cols].fillna(0.0)

# (Optional) If you prefer to keep NaN and only fill in derived metrics,
# comment out the two lines above.

# ------------------------------------------------------------
# 6) Save the merged, cleaned dataset
# ------------------------------------------------------------

df.to_csv("../data/merged_cia_countries.csv", index=False)
print("Merged data shape:", df.shape)
print("Saved to data/merged_cia_countries.csv")


Merged data shape: (259, 72)
Saved to data/merged_cia_countries.csv


In [ ]:
mydata = pd.read_csv("../data/merged_cia_countries.csv")
print("Data loaded...")

Data loaded...


Starting plotting for dashboard, as well as some data checks and investigation

In [ ]:
mydata.head()

,Country,Real_GDP_PPP_billion_USD,GDP_Official_Exchange_Rate_billion_USD,Real_GDP_Growth_Rate_percent,Real_GDP_per_Capita_USD,Unemployment_Rate_percent,Youth_Unemployment_Rate_percent,Budget_billion_USD,Budget_Surplus_billion_USD,Budget_Deficit_percent_of_GDP,...,Other_Land,Agricultural_Land,Arable_Land (%% of Total Agricultural Land),Permanent_Crops (%% of Total Agricultural Land),Permanent_Pasture (%% of Total Agricultural Land),Irrigated_Land,Capital,Capital_Coordinates,Government_Type,Suffrage_Age
0,AFGHANISTAN,60.80,20.24,-20.74,1500.0,13.28,20.2,5.09,15.1,-15.1,...,40.1,58.1,11.8,0.3,46.0,0.0,Kabul,"34 31 N, 69 11 E",Theocracy,18.0
1,AKROTIRI,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Episkopi Cantonment,"34 40 N, 32 51 E",NaN,0.0
2,ALBANIA,40.82,15.27,8.52,14500.0,11.82,27.8,4.19,2.0,-2.0,...,28.2,42.8,22.3,3.0,17.4,0.0,Tirana,"41 19 N, 19 49 E",Republic,18.0
3,ALGERIA,487.72,169.91,3.50,11000.0,12.70,31.9,55.19,9.6,-9.6,...,81.8,17.4,3.2,0.4,13.8,0.0,Algiers,"36 45 N, 3 03 E",Republic,18.0
4,AMERICAN SAMOA,658.00,658.00,-1.87,11200.0,29.80,0.0,249.00,2.1,-2.1,...,0.0,24.5,15.0,9.5,0.0,0.0,Pago Pago,"14 16 S, 170 42 W",Republic,18.0


In [ ]:
histogram = px.histogram(mydata, x="Real_GDP_per_Capita_USD")
histogram.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed